In [1]:
! sudo apt-get update
! sudo mkdir -p /usr/share/man/man1
! sudo apt-get install -y openjdk-11-jdk
! pip install pyspark

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian-security buster/updates InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease




openjdk-11-jdk is already the newest version (11.0.18+10-1~deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

from pyspark.sql.types import StructType, StructField # Estructura del dataframe
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql.types import Row # formatos para las columnas

In [3]:
spark = SparkContext(master='local', appName='udf')
sql_context = SQLContext(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/28 21:05:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/root/venv/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
!head -5 '/data/deportistaError.csv'

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,,,273
4,Edgar Lindenau Aabye,1,34,,,278


In [5]:
athletes_mistake_rdd = spark.textFile('/data/deportista.csv').map(
    lambda line: line.split(',')
)

In [6]:
def remove_header(index, iterator):
    return iter(list(iterator)[1:])

In [7]:
athletes_mistake_rdd = athletes_mistake_rdd.mapPartitionsWithIndex(remove_header)

In [8]:
athletes_mistake_rdd.take(2)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199']]

In [9]:
athletes_mistake_rdd = athletes_mistake_rdd.map(lambda l: (
    l[0],
    l[1],
    l[2],
    l[3],
    l[4],
    l[5],
    l[6]
    )
)

athletes_mistake_schema = StructType([
    StructField("deportista_id", StringType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", StringType(), False),
    StructField("edad", StringType(), False),
    StructField("altura", StringType(), False), 
    StructField("peso", StringType(), False),
    StructField("equipo_id", StringType(), False)
])
athletes_mistake_df = sql_context.createDataFrame(athletes_mistake_rdd, athletes_mistake_schema)

In [10]:
athletes_mistake_df.show()

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|  80|      199|
|            2|            A Lamusi|     1|  23|   170|  60|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0|   0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0|   0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|            6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|            7|        John Aalberg|     1|  31|   183|  72|     1096|
|            8|Cornelia Cor Aalt...|     2|  18|   168|   0|      705|
|            9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
|           10|Einar Ferdinand E...|     1|  26|     0|   0|      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
|     

In [11]:
from pyspark.sql.functions import udf

In [12]:
def convert_integer(value):
    return int(value) if int(value) > 0 else None

In [13]:
convert_integer_udf = udf(lambda z: convert_integer(z), IntegerType()) # crear la udf a partir de la función
sql_context.udf.register("convert_integer_udf", convert_integer_udf) # dar de alta la udf

<function __main__.<lambda>(z)>

In [15]:
athletes_mistake_df.select(convert_integer_udf("altura").alias("alturaUDF")).show()

+---------+
|alturaUDF|
+---------+
|      180|
|      170|
|     null|
|     null|
|      185|
|      188|
|      183|
|      168|
|      186|
|     null|
|      182|
|      172|
|      159|
|      171|
|     null|
|      184|
|      175|
|      189|
|     null|
|      176|
+---------+
only showing top 20 rows

